<a href="https://colab.research.google.com/github/ramee-1001/ADless/blob/main/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

감성분석 예제


In [2]:
pip install konlpy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 51.4 MB/s 


In [7]:
pip install pytagcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
pip install pygame


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 6.0 MB/s 


In [10]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
# 지표를 하나만 설정할 경우
from sklearn.model_selection import cross_val_score
# 지표를 하나 이상 설정할 경우
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 모델의 최적의 하이퍼파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집화
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth



# ARIMA (시계열 예측)
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm


# 시간 측정을 위한 시간 모듈
import datetime

# 주식정보
from pandas_datareader import data

# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer


# 데이터 수집
import requests
from bs4 import BeautifulSoup
import re
import time
import os
import json

# 한국어 형태소 분석
from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran

# # 워드 클라우드를 위한 라이브러리
# from collections import Counter
# import pytagcloud
# from IPython.display import Image

# 저장
import pickle

In [12]:
df = pd.read_csv("/content/review_data.csv")
df

,score,review,y
0,5,친절하시고 깔끔하고 좋았습니다,1
1,5,조용하고 고기도 굿,1
2,4,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",1
3,4,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,1
4,5,고기도 맛있고 서비스는 더 최고입니다~,1
...,...,...,...
540,3,추웟어요 고기 외에는 별로에요..,0
541,1,고기질과 육전은 좋다.다만 한우손님 돼지고기 손님을 차별한다(돼지손님은 주차불가.네...,0
542,5,직접 구워주시고 진짜맛있음. 반찬도 맛있음. 직원분이 친절하게 잘해주시네요,1
543,4,친절하게 서빙해주시고 음식도 챙겨주셨어요 ㅎ,1


In [15]:
def text_clearing(text): 
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    # 지정한 정규식에 해당하지 않은 것은 길이가 0인 문자열로 변환한다.
  result = hangul.sub('', text)
    
  return result

In [16]:
df["ko_review"] = df["review"].apply(lambda x : text_clearing(x))

In [17]:
df.drop("review", axis=1, inplace=True)
df

,score,y,ko_review
0,5,1,친절하시고 깔끔하고 좋았습니다
1,5,1,조용하고 고기도 굿
2,4,1,갈비탕과 냉면 육회비빔밥이 맛있습니다
3,4,1,대체적으로 만족하나와인의 구성이 살짝 아쉬움
4,5,1,고기도 맛있고 서비스는 더 최고입니다
...,...,...,...
540,3,0,추웟어요 고기 외에는 별로에요
541,1,0,고기질과 육전은 좋다다만 한우손님 돼지고기 손님을 차별한다돼지손님은 주차불가네이버예...
542,5,1,직접 구워주시고 진짜맛있음 반찬도 맛있음 직원분이 친절하게 잘해주시네요
543,4,1,친절하게 서빙해주시고 음식도 챙겨주셨어요 ㅎ


In [18]:
# konlpy 라이브러리로 텍스트 데이터에서 형태소를 추출한다.
def get_pos (x) :
    tagger = Okt()
    pos = tagger.pos(x)
    
    # 단어와 품사를 합쳐서 하나의 단어로 만들어준다.
    result = []
    
    # 형태소의 수만큼 반복한다.
    # 조사인 것과 명사인 것이 같을 수 있기 때문에 구분해준다.
    # 형태소 벡터를 만들때 추후 사용
    for a1 in pos :
        result.append(f'{a1[0]}/{a1[1]}')
    
    return result

In [19]:
get_pos("동해물과 백두산이 마르고 닳도록")

['동/Modifier',
 '해물/Noun',
 '과/Josa',
 '백두산/Noun',
 '이/Josa',
 '마르고/Noun',
 '닳도록/Verb']

In [20]:
index_vectorizer = CountVectorizer(tokenizer=lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df["ko_review"].tolist())
X

<545x3030 sparse matrix of type '<class 'numpy.int64'>'
	with 9692 stored elements in Compressed Sparse Row format>

In [21]:
index_vectorizer.vocabulary_

{'친절하시고/Adjective': 2647,
 '깔끔하고/Adjective': 428,
 '좋았습니다/Adjective': 2403,
 '조용하고/Adjective': 2356,
 '고기/Noun': 233,
 '도/Josa': 721,
 '굿/Noun': 330,
 '갈비탕/Noun': 120,
 '과/Josa': 260,
 '냉면/Noun': 528,
 '육회/Noun': 2065,
 '비빔밥/Noun': 1419,
 '이/Josa': 2082,
 '맛있습니다/Adjective': 1013,
 '대/Modifier': 671,
 '체적/Noun': 2604,
 '으로/Josa': 2067,
 '만족하나/Adjective': 956,
 '와인/Noun': 1996,
 '의/Josa': 2077,
 '구성/Noun': 293,
 '살짝/Noun': 1476,
 '아쉬움/Noun': 1705,
 '맛있고/Adjective': 1001,
 '서비스/Noun': 1508,
 '는/Josa': 589,
 '더/Noun': 701,
 '최고/Noun': 2613,
 '입니다/Adjective': 2182,
 '가/Josa': 24,
 '입/Noun': 2177,
 '에서/Josa': 1897,
 '녹아요/Verb': 553,
 '였습니다/Verb': 1935,
 '살살/Noun': 1473,
 '녹는/Verb': 552,
 '최상급/Noun': 2615,
 '소고기/Noun': 1540,
 '를/Josa': 901,
 '맛/Noun': 995,
 '보고왔습니다/Verb': 1316,
 '구워주고/Verb': 302,
 '가성/Noun': 73,
 '비/Noun': 1408,
 '짱/Noun': 2528,
 '콜키/Noun': 2688,
 '지/Josa': 2477,
 '프리/Noun': 2788,
 '라서/Josa': 885,
 '을/Josa': 2070,
 '가지/Noun': 89,
 '고/Josa': 227,
 '가면/Noun': 61,
 '좋은/Adjective

In [22]:
# 첫번째 행의 리뷰 이름
print(df["ko_review"][0])
# 첫번째 행의 긍정/부정 시그널
print(df["y"][0])
# 첫번째 행의 index_vectorizer의 (형태소, 시리얼넘버)  
print(X[0])

친절하시고 깔끔하고 좋았습니다
1
  (0, 2647)	1
  (0, 428)	1
  (0, 2403)	1


In [24]:
# TFidf 변환 모델 생성
tfidf_vectorizer = TfidfTransformer()
# 형태소 벡터 변환하기
X = tfidf_vectorizer.fit_transform(X)
print(X[0])

  (0, 2647)	0.5548708693511647
  (0, 2403)	0.48955631270748484
  (0, 428)	0.6726462183300624


In [25]:
y = df["y"]

In [26]:
# KNN
params = {
	"n_neighbors" : list(range(1, 11))
}

# 사용할 모델 객체 생성
model1 = KNeighborsClassifier()

# 최적의 하이퍼 파라미터를 찾는다. 
kfold = KFold(n_splits=10, shuffle=True, random_state = 1)
grid_clf1 = GridSearchCV(model1, param_grid = params, scoring="f1", cv=kfold)
grid_clf1.fit(X, y)

# 결과 출력
print(f"최적의 하이퍼 파라미터 : {grid_clf1.best_params_}")
print(f"최적의 모델 평균 성능 : {grid_clf1.best_score_}")

최적의 하이퍼 파라미터 : {'n_neighbors': 4}
최적의 모델 평균 성능 : 0.9561087805947686


In [27]:
# LogisticRegression 
# penalty : 규제의 종류(l1, l2, elasticnet, none)
# C : 규제의 강도 
params = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

model2 = LogisticRegression()

grid_clf2 = GridSearchCV(model2, param_grid=params, scoring='f1', cv=kfold)
grid_clf2.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf2.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf2.best_score_}')

최적의 하이퍼 파라미터 : {'C': 0.0001, 'penalty': 'none'}
최적의 모델 평균 성능 : 0.9580627713400792


In [28]:
# SVM(SVC)
# SVM은 penalty가 l2로 고정되어 있다
# C : 규제의 강도 
params = {
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

model3 = SVC()

grid_clf3 = GridSearchCV(model3, param_grid=params, scoring='f1', cv=kfold)
grid_clf3.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf3.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf3.best_score_}')

최적의 하이퍼 파라미터 : {'C': 0.0001}
최적의 모델 평균 성능 : 0.9479277809113741


In [29]:
# RandomForest
# n_estimators : 사용할 트리의 개수
# max_depth : 생성될 최대 질문 깊이, None은 무한대.
params = {
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'max_depth' : [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

model4 = RandomForestClassifier(random_state=1)

grid_clf4 = GridSearchCV(model4, param_grid=params, scoring='f1', cv=kfold)
grid_clf4.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf4.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf4.best_score_}')

최적의 하이퍼 파라미터 : {'max_depth': None, 'n_estimators': 50}
최적의 모델 평균 성능 : 0.9479277809113741


In [30]:
# XGBoost
# booster : 내부에 사용할 알고리즘
# learning_rate : 학습률
# n_estimators : 사용할 트리의 개수
# max_depth : 생성될 최대 질문 깊이, None은 무한대.

params = {
    'booster' : ['gbtree', 'gblinear'],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    # 'max_depth' : [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

model5 = XGBClassifier(silent=True, verbosity=0)

grid_clf5 = GridSearchCV(model5, param_grid=params, scoring='f1', cv=kfold)
grid_clf5.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf5.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf5.best_score_}')

최적의 하이퍼 파라미터 : {'booster': 'gbtree', 'learning_rate': 0.1, 'n_estimators': 100}
최적의 모델 평균 성능 : 0.9504665890655344


In [31]:
print(grid_clf1.best_score_)
print(grid_clf2.best_score_)
print(grid_clf3.best_score_)
print(grid_clf4.best_score_)
print(grid_clf5.best_score_)

0.9561087805947686
0.9580627713400792
0.9479277809113741
0.9479277809113741
0.9504665890655344


In [32]:
model = grid_clf2.best_estimator_

# 상관관계수 구하기
a1 = (model.coef_[0])
display(a1)
len(a1)

array([ 1.62588263,  5.9313457 ,  4.33385186, ...,  0.47754467,
       -9.27302276,  0.90485881])

3030

In [33]:
a2 = list(enumerate(a1))
a2

[(0, 1.6258826263307251),
 (1, 5.931345700126292),
 (2, 4.3338518629697),
 (3, 2.385906465895489),
 (4, 1.8653806263182187),
 (5, 1.4046298707331188),
 (6, 4.960703969939797),
 (7, 9.668211340430336),
 (8, 3.5546579361654023),
 (9, 1.1470427287642773),
 (10, 0.704284260723358),
 (11, 0.6028054248027078),
 (12, 0.7615770091676071),
 (13, 0.5430034540339932),
 (14, -5.776360160743133),
 (15, 5.117483504013158),
 (16, -16.8777551464071),
 (17, 1.8984202284912244),
 (18, -23.953359513506584),
 (19, 0.5305659753809535),
 (20, 1.3260447662579673),
 (21, 0.9844892670008357),
 (22, 1.0017194148830226),
 (23, -2.084336924543278),
 (24, -17.170331441520453),
 (25, -3.471194717689412),
 (26, 0.25687748780728853),
 (27, -4.186025217694819),
 (28, 0.6028054248027078),
 (29, 0.3533190488957657),
 (30, 7.202868887670492),
 (31, 0.00859817727189931),
 (32, 1.3025815763478008),
 (33, 2.3271700420463124),
 (34, -5.409565076418159),
 (35, 0.37992783106005323),
 (36, 0.8071915482407522),
 (37, 0.543034901

In [34]:
a3 = []

for idx, value in a2 : 
	a3.append((value, idx))
    
a3

[(1.6258826263307251, 0),
 (5.931345700126292, 1),
 (4.3338518629697, 2),
 (2.385906465895489, 3),
 (1.8653806263182187, 4),
 (1.4046298707331188, 5),
 (4.960703969939797, 6),
 (9.668211340430336, 7),
 (3.5546579361654023, 8),
 (1.1470427287642773, 9),
 (0.704284260723358, 10),
 (0.6028054248027078, 11),
 (0.7615770091676071, 12),
 (0.5430034540339932, 13),
 (-5.776360160743133, 14),
 (5.117483504013158, 15),
 (-16.8777551464071, 16),
 (1.8984202284912244, 17),
 (-23.953359513506584, 18),
 (0.5305659753809535, 19),
 (1.3260447662579673, 20),
 (0.9844892670008357, 21),
 (1.0017194148830226, 22),
 (-2.084336924543278, 23),
 (-17.170331441520453, 24),
 (-3.471194717689412, 25),
 (0.25687748780728853, 26),
 (-4.186025217694819, 27),
 (0.6028054248027078, 28),
 (0.3533190488957657, 29),
 (7.202868887670492, 30),
 (0.00859817727189931, 31),
 (1.3025815763478008, 32),
 (2.3271700420463124, 33),
 (-5.409565076418159, 34),
 (0.37992783106005323, 35),
 (0.8071915482407522, 36),
 (0.5430349012331

In [35]:
coef_pos_index = sorted(a3, reverse=True)
coef_pos_index

[(37.752909139298026, 1017),
 (26.8870472229759, 1001),
 (26.516040494397743, 2246),
 (21.613079296687232, 1030),
 (21.05925257549977, 1093),
 (18.889134518202987, 999),
 (16.953422188475255, 2376),
 (16.69560949704242, 721),
 (16.11493001586533, 1388),
 (15.367162571960328, 2613),
 (15.013308941497638, 2388),
 (14.840877126002361, 1029),
 (14.087713804318113, 2330),
 (13.069147688498388, 2647),
 (12.650446735195754, 608),
 (12.50729998020483, 2404),
 (12.418365819411102, 2247),
 (12.072419753448433, 2128),
 (11.837163425405564, 873),
 (11.366648285114866, 1006),
 (11.243437975376176, 1008),
 (11.152692131917735, 2502),
 (11.070654775932596, 961),
 (10.657225361616705, 1386),
 (10.269000408105459, 635),
 (10.065231520668108, 2125),
 (9.971587516017907, 897),
 (9.72582472246426, 2363),
 (9.668211340430336, 7),
 (9.384206523381321, 607),
 (9.32490807139145, 945),
 (9.23967618161695, 1007),
 (9.225130386911387, 428),
 (9.007354616747493, 1137),
 (8.875140191928525, 1467),
 (8.567577820859

In [36]:
# 새로운 딕셔너리 생성
text_data_dict = {}

# 단어 사전에 있는 단어의 수만큼 반복한다.
for key in index_vectorizer.vocabulary_ :
	# 현재 key에 해당하는 값을 가져온다.
    value = index_vectorizer.vocabulary_[key]
    
    # 위의 딕셔너리에 담는다.
    text_data_dict[value] = key
    
text_data_dict

{2647: '친절하시고/Adjective',
 428: '깔끔하고/Adjective',
 2403: '좋았습니다/Adjective',
 2356: '조용하고/Adjective',
 233: '고기/Noun',
 721: '도/Josa',
 330: '굿/Noun',
 120: '갈비탕/Noun',
 260: '과/Josa',
 528: '냉면/Noun',
 2065: '육회/Noun',
 1419: '비빔밥/Noun',
 2082: '이/Josa',
 1013: '맛있습니다/Adjective',
 671: '대/Modifier',
 2604: '체적/Noun',
 2067: '으로/Josa',
 956: '만족하나/Adjective',
 1996: '와인/Noun',
 2077: '의/Josa',
 293: '구성/Noun',
 1476: '살짝/Noun',
 1705: '아쉬움/Noun',
 1001: '맛있고/Adjective',
 1508: '서비스/Noun',
 589: '는/Josa',
 701: '더/Noun',
 2613: '최고/Noun',
 2182: '입니다/Adjective',
 24: '가/Josa',
 2177: '입/Noun',
 1897: '에서/Josa',
 553: '녹아요/Verb',
 1935: '였습니다/Verb',
 1473: '살살/Noun',
 552: '녹는/Verb',
 2615: '최상급/Noun',
 1540: '소고기/Noun',
 901: '를/Josa',
 995: '맛/Noun',
 1316: '보고왔습니다/Verb',
 302: '구워주고/Verb',
 73: '가성/Noun',
 1408: '비/Noun',
 2528: '짱/Noun',
 2688: '콜키/Noun',
 2477: '지/Josa',
 2788: '프리/Noun',
 885: '라서/Josa',
 2070: '을/Josa',
 89: '가지/Noun',
 227: '고/Josa',
 61: '가면/Noun',
 2410: '좋은/Adj

In [37]:
# 긍정적인 어조 (상관계수가 1에 가장 큰)
top20 = coef_pos_index[:20]
# 부정적인 어조
bottom20 = coef_pos_index[-20:]

In [38]:
for value, idx in top20 : 
	print(text_data_dict[idx])

맛있어요/Adjective
맛있고/Adjective
잘/Verb
맛있었어요/Adjective
먹었습니다/Verb
맛있게/Adjective
좋고/Adjective
도/Josa
분위기/Noun
최고/Noun
좋아요/Adjective
맛있었습니다/Adjective
정말/Noun
친절하시고/Adjective
다/Adverb
좋았어요/Adjective
잘/VerbPrefix
이에요/Josa
또/Noun
맛있네요/Adjective


In [39]:
for value, idx in bottom20 : 
	print(text_data_dict[idx])

추웟어/Noun
한참/Noun
체인점/Noun
ㅠㅠ/KoreanParticle
가/Josa
주문/Noun
는/Josa
분/Noun
종업원/Noun
굿/Noun
적어요/Verb
를/Josa
시끄러워요/Adjective
ㅠㅠㅠㅠ/KoreanParticle
점/Noun
을/Josa
은/Josa
기대이하였음/Verb
많이/Adverb
별로/Noun
